In [1]:
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Load data from .npz file
data = np.load('../prediction_model/features/symptoms.npz')
features = data['X']
labels = data['y']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Define the parameter grid to search for Logistic Regression
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'solver': ['liblinear', 'lbfgs', 'newton-cg', 'sag', 'saga'],
}

# Create a Logistic Regression classifier
logreg = LogisticRegression(random_state=42)

# Create the GridSearchCV object
grid_search = GridSearchCV(logreg, param_grid, cv=3, scoring='accuracy', n_jobs=-1)

# Fit the GridSearchCV object to the data
grid_search.fit(X_train, y_train)

# Get the best parameters and the corresponding model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Make predictions on the test set using the best model
predictions = best_model.predict(X_test)

# Evaluate the best model
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print('Best Parameters:', best_params)
print(f'Best Model Accuracy: {accuracy}')
print('Classification Report:\n', report)


Best Parameters: {'C': 0.001, 'solver': 'liblinear'}
Best Model Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        25
           1       1.00      1.00      1.00        15
           2       1.00      1.00      1.00        26
           3       1.00      1.00      1.00        21
           4       1.00      1.00      1.00        24
           5       1.00      1.00      1.00        33
           6       1.00      1.00      1.00        19
           7       1.00      1.00      1.00        34
           8       1.00      1.00      1.00        22
           9       1.00      1.00      1.00        23
          10       1.00      1.00      1.00        23
          11       1.00      1.00      1.00        21
          12       1.00      1.00      1.00        18
          13       1.00      1.00      1.00        23
          14       1.00      1.00      1.00        26
          15       1.00      1.00